## Load the Data

In [ ]:
# 필요한 모든 라이브러리 임포트
import numpy as np
import pandas as pd
import random
import networkx as nx
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

# CORA 데이터셋 로드
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# torch_geometric의 데이터를 NetworkX 그래프로 변환
edge_index = data.edge_index
edges = edge_index.t().numpy()
G = nx.from_edgelist(edges, create_using=nx.Graph())

## Random Walk

In [ ]:
 def random_walk(G, start_node, walk_length):
    walk = [start_node] # 시작 노드를 포함하는 리스트 walk를 생성
    # walk 리스트는 무작위 경로의 노드를 저장하는 데 사용
    
    for i in range(walk_length - 1):
        # 시작 노드가 이미 walk안에 있으니, -1.
        neighbors = list(G.neighbors(walk[-1]))
        if len(neighbors) == 0:
            break
        next_node = random.choice(neighbors)
        walk.append(next_node)
    return walk

In [ ]:
 print(random_walk(G,0,20))

## Deep Walk

In [ ]:
 def unbiased_random_walk(G, node, walk_length):
    walk = [node]
    for _ in range(walk_length - 1):
        neighbors = list(G.neighbors(walk[-1]))
        if len(neighbors) == 0:
            break
        walk.append(random.choice(neighbors))
    return walk

# Random Walk 생성
walks = [unbiased_random_walk(G, node, 10) for node in range(data.num_nodes) for _ in range(10)]

In [ ]:
 # String 형태로 변환 (Word2Vec 입력을 위해)
walks = [[str(node) for node in walk] for walk in walks]

# Word2Vec 학습
model = Word2Vec(walks, vector_size=100, window=5, min_count=0, hs=1, sg=1, workers=4, epochs=10)

# 노드 임베딩 추출
embeddings = np.array([model.wv.get_vector(str(i)) for i in range(data.num_nodes)])

In [ ]:
 # t-SNE를 통한 시각화
embeddings_2d = TSNE(n_components=2).fit_transform(embeddings)
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=data.y)
plt.show()

In [ ]:
 # 레이블 추출
labels = data.y.numpy()

# 노드 선택 (예: 첫 50개 노드)
selected_nodes = range(500)
subG = G.subgraph(selected_nodes)

# 레이블에 따른 색상 매핑
label_color_mapping = {label: idx for idx, label in enumerate(np.unique(labels[selected_nodes]))}
node_colors = [label_color_mapping[label] for label in labels[selected_nodes]]

plt.figure(figsize=(12,12), dpi=300)
plt.axis('off')
nx.draw_networkx(subG,
                 pos=nx.spring_layout(subG, seed=42, k=0.15),  # k 인자를 조정하여 노드 간의 거리를 늘릴 수 있습니다.
                 node_color=node_colors,
                 node_size=400,  # 노드의 크기를 줄입니다.
                 edge_color='black',  # 엣지의 색상을 변경합니다.
                 width=1.5,  # 엣지의 두께를 두꺼워지도록 설정합니다.
                 cmap='coolwarm',
                 font_size=14,
                 font_color='black')
plt.show()

## Node Classification

In [ ]:
# 레이블이 있는 노드만 선택
labels = data.y.numpy()
idx_train = data.train_mask.numpy()
idx_test = data.test_mask.numpy()

X_train, y_train = embeddings[idx_train], labels[idx_train]
X_test, y_test = embeddings[idx_test], labels[idx_test]

# 랜덤 포레스트 분류기 학습
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

## final plotting

In [ ]:
 # 하나, 둘, 셋, 치즈! 📸

plt.figure(figsize= (12,12), dpi= 300)
plt.axis('off')
nx.draw_networkx(G,
               pos = nx.spring_layout(G, seed = 0),
               node_color = labels,
               node_size = 300,  # 노드 크기 조정
               cmap = 'coolwarm',
               font_size = 14,
               font_color = 'white',
               edge_color = 'grey',  # 엣지 색상 설정
               width = 1,  # 엣지 두께 설정
               with_labels = False)  # 노드 라벨 표시 여부. True로 바꾸면 노드 이름이 출력됩니다!